In [79]:
import numpy as np
import pandas as pd
import subprocess

import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
asas_dir = "/Users/adamamiller/astronomy/ASAS/"
lc_dir = asas_dir + "ACVS_light_curves/"
cat_dir = asas_dir + "catI/ASASCatalog/catalog/"

In [3]:
macc = pd.read_csv(cat_dir + "macc_all_features_20150605.csv")

In [233]:
sum((macc["P_Class"] > 0.97) & (macc["Class"] == 'RR_Lyrae_FM'))

614

In [206]:
np.unique(macc["Class"], return_counts=True)

(array(['Beta_Cephei', 'Beta_Lyrae', 'Beta_Persei', 'ChemPeculiar',
        'ClassT_Tauri', 'Classical_Cepheid', 'Delta_Scuti', 'Ellipsoidal',
        'Herbig_AEBE', 'LSP', 'Mira', 'MultiMode_Cepheid', 'PopII_Cepheid',
        'Pulsating_Be', 'RCB', 'RR_Lyrae_DM', 'RR_Lyrae_FM', 'RR_Lyrae_FO',
        'RSG', 'RS_CVn', 'RV_Tauri', 'SARG_A', 'SARG_B', 'SX_Phe',
        'S_Doradus', 'Semireg_PV', 'W_Ursae_Maj', 'Weakline_T_Tauri'], dtype=object),
 array([ 259, 1963, 2855,  345,    5,  568, 1527,    2,  111, 5096, 3086,
         202,   54,  337,   53,  130, 1460,  476,  827,  263,  452, 3974,
        7820,   12,    1, 9982, 6025, 2239]))

In [262]:
training_set = np.where( 
                         (((macc["P_Class"] > 0.9975) & 
                          ( (macc["Class"] == 'Mira'))) | 
                          ((macc["P_Class"] > 0.975) &
                          ( (macc["Class"] == 'W_Ursae_Maj') 
                            ) ) | 
                          ((macc["P_Class"] > 0.96) &
                          ( (macc["Class"] == 'RR_Lyrae_FM') |
                            (macc["Class"] == 'Beta_Persei')) ) |
                          ((macc["P_Class"] > 0.84) & 
                          ( (macc["Class"] == 'Beta_Lyrae') |
                            (macc["Class"] == 'Classical_Cepheid')
                            )) | 
                         
                         # include the RCB sources
                          ((macc["P_Class"] > 0.8) & 
                          (macc["Class"] == 'RCB'))) & 
                         # throw out bad RA sources
                          ((macc["RA"] < 10*15) | 
                          (macc["RA"] >= 11*15))
                        
                       )
len(training_set[0])

2992

In [263]:
np.unique(macc["Class"].iloc[training_set], return_counts=True)

(array(['Beta_Lyrae', 'Beta_Persei', 'Classical_Cepheid', 'Mira', 'RCB',
        'RR_Lyrae_FM', 'W_Ursae_Maj'], dtype=object),
 array([459, 236, 219, 619,  16, 687, 756]))

In [264]:
from sklearn.model_selection import StratifiedShuffleSplit

In [265]:
rs = 23
columns = ["ASAS_ID", "Class", "P_Class"]
master = macc[columns].iloc[training_set]

master["Class"]

master["Class"] = master["Class"].map({'RCB': 'RCB', 
                                       'Mira': 'Mira', 
                                       'RR_Lyrae_FM': 'RRL', 
                                       'Beta_Persei': 'dEB', 
                                       'Classical_Cepheid': 'Cepheid',
                                       'Beta_Lyrae': 'aEB',
                                       'W_Ursae_Maj': 'WU'})
master_y = np.array(master["Class"])
dummy_X = np.arange(len(master_y)).reshape(len(master_y),1)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.66, random_state=rs)
sss.get_n_splits(dummy_X, master_y)
print(sss)
sss.split(dummy_X, master_y)

StratifiedShuffleSplit(n_splits=1, random_state=23, test_size=0.66,
            train_size=None)


<generator object BaseShuffleSplit.split at 0x119f1de60>

In [266]:
for train_idx, test_idx in sss.split(dummy_X, master_y):
    master_train = master.iloc[train_idx]
    master_test = master.iloc[test_idx]

In [271]:
# for asas_id in master_train["ASAS_ID"]:
#     subprocess.Popen(['cp', lc_dir+asas_id, './training_lcs/'])

In [272]:
# for asas_id in master_test["ASAS_ID"]:
#     subprocess.Popen(['cp', lc_dir+asas_id, './test_lcs/'])

In [273]:
macc[['ASAS_ID', 'Class', 'P', 'V']].iloc[training_set[0][train_idx]]

,ASAS_ID,Class,P,V
43769,195044-7146.3,Mira,243.658116,8.07
46261,203827+0912.1,RR_Lyrae_FM,0.580895,12.76
30944,170013-2623.9,W_Ursae_Maj,0.434035,12.56
25245,143730-4422.1,Mira,295.785439,11.91
1000,014231-8000.6,RR_Lyrae_FM,0.348376,12.09
24597,141748-5311.2,Beta_Lyrae,1.514158,11.12
35966,180847-8647.9,Mira,256.645368,8.78
42786,193546-1136.3,W_Ursae_Maj,0.424015,11.51
46369,204100-6705.3,RR_Lyrae_FM,0.657169,13.35
3748,050909-7707.9,W_Ursae_Maj,0.369323,12.80


In [292]:
macc[['ASAS_ID', 'Class', 'P', 'V']].iloc[[30423, 28508, 17258, 49268, 2613, 27396, 47488]]

,ASAS_ID,Class,P,V
30423,164912-3246.0,Beta_Persei,0.742595,9.12
28508,160448-4520.4,Beta_Persei,2.222008,10.30
17258,095841-1104.4,Beta_Persei,1.323355,9.10
49268,224745-2915.6,Beta_Persei,0.691779,10.71
2613,040907-4828.6,Beta_Persei,1.191648,8.52
27396,153835-6727.8,Beta_Persei,1.107174,11.06
47488,211419-4247.9,Beta_Persei,2.695932,8.37


In [167]:
macc[['ASAS_ID', 'Class', 'P', 'V']].iloc[[39251, 29735, 48587, 41068]]

,ASAS_ID,Class,P,V
39251,184732-3809.6,RCB,2392.181621,9.78
29735,163242-5315.6,RCB,1766.149384,10.69
48587,220320-1637.6,RCB,1541.980354,11.62
41068,191012-2029.7,RCB,1237.556556,11.96


In [276]:
# write out the training file
master_train[["ASAS_ID", "Class"]].to_csv("training_sources.csv", index=False)
master_test[["ASAS_ID", "Class"]].to_csv("test_sources.csv", index=False)

In [293]:
def read_lc(filename):
    hjd, mag, mag_unc = np.loadtxt(filename, unpack=True)
    return hjd, mag, mag_unc

In [295]:
def calc_feature(df, train=True):
    
    if train==True:
        lc_dir = "./training_lcs/"
    else:
        lc_dir = "./test_lcs/"
    
    feature = np.empty(len(df))
    for source_num, asas_id in enumerate(df["ASAS_ID"]):
        t, m, mu = read_lc(lc_dir+asas_id)
        # feature calculations
        # feature calculations
        # feature calculations
        
        feature[source_num] = feat_val
    
    return feature

In [296]:
def calc_mean(df, train=True):
    
    if train==True:
        lc_dir = "./training_lcs/"
    else:
        lc_dir = "./test_lcs/"
    
    feature = np.empty(len(df))
    for source_num, asas_id in enumerate(df["ASAS_ID"]):
        t, m, mu = read_lc(lc_dir+asas_id)
        
        feat_val = np.mean(m)
        
        feature[source_num] = feat_val
    
    return feature

In [297]:
def calc_median(df, train=True):
    
    if train==True:
        lc_dir = "./training_lcs/"
    else:
        lc_dir = "./test_lcs/"
    
    feature = np.empty(len(df))
    for source_num, asas_id in enumerate(df["ASAS_ID"]):
        t, m, mu = read_lc(lc_dir+asas_id)
        
        feat_val = np.median(m)
        
        feature[source_num] = feat_val
    
    return feature

In [298]:
def calc_std(df, train=True):
    
    if train==True:
        lc_dir = "./training_lcs/"
    else:
        lc_dir = "./test_lcs/"
    
    feature = np.empty(len(df))
    for source_num, asas_id in enumerate(df["ASAS_ID"]):
        t, m, mu = read_lc(lc_dir+asas_id)
        
        feat_val = np.std(m, ddof=1)
        
        feature[source_num] = feat_val
    
    return feature

In [299]:
def calc_nobs(df, train=True):
    
    if train==True:
        lc_dir = "./training_lcs/"
    else:
        lc_dir = "./test_lcs/"
    
    feature = np.empty(len(df))
    for source_num, asas_id in enumerate(df["ASAS_ID"]):
        t, m, mu = read_lc(lc_dir+asas_id)
        
        feat_val = len(t)
        
        feature[source_num] = feat_val
    
    return feature

In [300]:
def calc_duration(df, train=True):
    
    if train==True:
        lc_dir = "./training_lcs/"
    else:
        lc_dir = "./test_lcs/"
    
    feature = np.empty(len(df))
    for source_num, asas_id in enumerate(df["ASAS_ID"]):
        t, m, mu = read_lc(lc_dir+asas_id)
        
        feat_val = np.ptp(t)
        
        feature[source_num] = feat_val
    
    return feature

In [294]:
train_df = pd.read_csv("training_sources.csv")

In [301]:
train_df = train_df[["ASAS_ID", "Class"]]
feat_mean = calc_mean(train_df)
feat_median = calc_median(train_df)
feat_std = calc_std(train_df)
feat_nobs = calc_nobs(train_df)
feat_duration = calc_duration(train_df)

In [302]:
train_df['mean'] = feat_mean
# train_df['median'] = feat_median
# train_df['std'] = feat_std
train_df['nobs'] = feat_nobs
train_df['duration'] = feat_duration

In [308]:
train_df.to_csv("training_sources.csv", index=False)

In [309]:
test_df = pd.read_csv("test_sources.csv")
test_df = test_df[["ASAS_ID", "Class"]]
feat_mean = calc_mean(test_df, train=False)
feat_median = calc_median(test_df, train=False)
feat_std = calc_std(test_df, train=False)
feat_nobs = calc_nobs(test_df, train=False)
feat_duration = calc_duration(test_df, train=False)

In [310]:
test_df['mean'] = feat_mean
# test_df['median'] = feat_median
# test_df['std'] = feat_std
test_df['nobs'] = feat_nobs
test_df['duration'] = feat_duration

In [311]:
test_df.to_csv("test_sources.csv", index=False)